In [10]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local").setAppName("spark_sql_basic1")
sc   = SparkContext(conf=conf)

24/12/10 13:28:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
sc.stop()

In [11]:
movies_rdd = sc.parallelize([
    (1, ("어벤져스", "마블")),
    (2, ("슈퍼맨", "DC")),
    (3, ("배트맨", "DC")),
    (4, ("겨울왕국", "디즈니")),
    (5, ("아이언맨", "마블"))
])


attendances_rdd = sc.parallelize([
    (1, (13934592, "KR")),
    (2, (2182227,"KR")),
    (3, (4226242, "KR")),
    (4, (10303058, "KR")),
    (5, (4300365, "KR"))
])

In [ ]:
#관객수가 500만 이상인 영화를 가져오기 
1. 조인 > 필터 > 영화정보
2. 필터 > 조인 > 영화정보

In [12]:
#1. 조인먼저
movies_att = movies_rdd.join(attendances_rdd)
movies_att.filter(
    lambda x: x[1][1][0] > 5000000
).collect()

[(4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [ ]:
#2. 필터먼저

In [13]:
filtered_att = attendances_rdd.filter(lambda x: x[1][0]>5000000)
movies_rdd.join(filtered_att).collect()

[(4, (('겨울왕국', '디즈니'), (10303058, 'KR'))),
 (1, (('어벤져스', '마블'), (13934592, 'KR')))]

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark_sql_basic").getOrCreate()

In [15]:
movies = [
    (1, "어벤져스", "마블", 2012, 4, 26),
    (2, "슈퍼맨", "DC", 2013, 6, 13),
    (3, "배트맨", "DC", 2008, 8, 6),
    (4, "겨울왕국", "디즈니", 2014, 1, 16),
    (5, "아이언맨", "마블", 2008, 4, 30)
]

In [16]:
movie_schema = ['id', 'name', 'company', 'year', 'month', 'day']

# 1. DataFrame 만들기

In [18]:
df = spark.createDataFrame(data=movies, schema= movie_schema)

In [20]:
df.dtypes

[('id', 'bigint'),
 ('name', 'string'),
 ('company', 'string'),
 ('year', 'bigint'),
 ('month', 'bigint'),
 ('day', 'bigint')]

In [21]:
df.show()

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



In [23]:
df.select('name').show()

+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [24]:
df.filter(df.year>=2000).show() #col 객체

+---+--------+-------+----+-----+---+
| id|    name|company|year|month|day|
+---+--------+-------+----+-----+---+
|  1|어벤져스|   마블|2012|    4| 26|
|  2|  슈퍼맨|     DC|2013|    6| 13|
|  3|  배트맨|     DC|2008|    8|  6|
|  4|겨울왕국| 디즈니|2014|    1| 16|
|  5|아이언맨|   마블|2008|    4| 30|
+---+--------+-------+----+-----+---+



# SQL을 사용하기 위해서 View 등록

In [28]:
df.createOrReplaceTempView('movies')  #view name, table name

In [29]:
#view 를 select projection하는 문장

query = '''
select name from movies
'''

spark.sql(query).show()


+--------+
|    name|
+--------+
|어벤져스|
|  슈퍼맨|
|  배트맨|
|겨울왕국|
|아이언맨|
+--------+



In [30]:
#2010 년 이후 개봉한 영화 조회

query = '''
select name, company
from movies
where year>=2010
'''

In [31]:
spark.sql(query).show()

+--------+-------+
|    name|company|
+--------+-------+
|어벤져스|   마블|
|  슈퍼맨|     DC|
|겨울왕국| 디즈니|
+--------+-------+



In [ ]:
#회사가 마블인 영화 목록
where company = '마블'

In [ ]:
#~맨
where name like '%맨'


In [ ]:
# ~이~
where name like '%이%'

In [ ]:
#개봉월이 4~8월 사이
where month between 4 and 8

In [ ]:
# and, or 
# ~맨, 개봉연도가 2010년 이하
where name like '%맨' and
year <= 2010

In [ ]:
#회사가 마블, dc 인 영화
where company = '마블' or company = 'DC'
where company in ('마블','DC')


In [ ]:
# 회사가 "마"로 시작을 하거나, "니"로 끝나는 영화
where company like '마%' or company like '%니'

In [ ]:
# 회사가 "마"로 시작을 하거나, "니"로 끝나는 영화 중 2010년 이후로 개봉한 영화
where (company like '마%' or company like '%니') 
and year >= 2010

ASC : ascending 오름차순, default
DESC : descending 내림차순

In [ ]:
# 개봉 연도 오름차순으로 확인
from movies
order by year ASC

In [ ]:
# count(*),  count(name)
# mean, sum
select count(*)

In [36]:
attendances = [
    [1, 13934592., "KR"],
    (2, 2182227.,"KR"),
    (3, 4226242., "KR"),
    (4, 10303058., "KR"),
    (5, 4300365., "KR")
]

In [37]:
from pyspark.sql.types import StringType, IntegerType, FloatType, StructType, StructField

att_schema = StructType([ # 모든 컬럼의 타입을 통칭 - 컬럼 데이터의 집합
    StructField("id", IntegerType(), True), # StructField : 컬럼
    StructField("att", FloatType(), True),
    StructField("theater_country", StringType(), True)
])

In [38]:
att_df = spark.createDataFrame( data=attendances, schema = att_schema )
att_df.dtypes

[('id', 'int'), ('att', 'float'), ('theater_country', 'string')]

In [39]:
att_df.show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [40]:
att_df.createOrReplaceTempView('att')

In [41]:
att_df.select('*').show()

+---+-----------+---------------+
| id|        att|theater_country|
+---+-----------+---------------+
|  1|1.3934592E7|             KR|
|  2|  2182227.0|             KR|
|  3|  4226242.0|             KR|
|  4|1.0303058E7|             KR|
|  5|  4300365.0|             KR|
+---+-----------+---------------+



In [48]:
query = '''
select movies.id, movies.name, movies.company, att.att, att.theater_country
from movies JOIN att ON  movies.id = att.id
'''

In [50]:
moviews_views = spark.sql(query)

In [51]:
moviews_views.createOrReplaceTempView('movies_view')

In [52]:
query = '''
select * from movies_view
'''

In [53]:
spark.sql(query).show()

+---+--------+-------+-----------+---------------+
| id|    name|company|        att|theater_country|
+---+--------+-------+-----------+---------------+
|  5|아이언맨|   마블|  4300365.0|             KR|
|  1|어벤져스|   마블|1.3934592E7|             KR|
|  3|  배트맨|     DC|  4226242.0|             KR|
|  2|  슈퍼맨|     DC|  2182227.0|             KR|
|  4|겨울왕국| 디즈니|1.0303058E7|             KR|
+---+--------+-------+-----------+---------------+



In [ ]:
query = '''
select * 
from movies_view
where company = 

'''

In [54]:
spark.stop()